## Поиск норм

In [3]:
import pandas as pd
import pymongo as pm
import elasticsearch

In [4]:
import sys
sys.path.append('..')
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
import lib.obo
import lib.classification.elastic
import lib.classification.validation
import lib.utils
import data
import pymonad

reload(lib.classification.validation)
reload(lib.classification.elastic)
reload(lib.obo)
reload(lib.utils)
reload(data)

from lib.classification.elastic import annotate_index, collapse_matches, build_synonyms_graph, search_term
from lib.utils import map_series, expand_dataframe, expand, flatten
from lib.classification.validation import correct_ratio, incorrect_ratio, perfect_match
from geo_annotation.import_ontology_es import import_ontology
from elasticsearch import Elasticsearch
from lib.obo import read_ontology, Ontology
pd.set_option('display.width', 512)

In [5]:
es = elasticsearch.Elasticsearch()

In [6]:
normal_terms = ['normal', 'control', 'reference', 'healthy']

In [5]:
norms = search_term(es=es,
                    term=normal_terms,
                    index='samples',
                    id_field='accession',
                    fields=['title', 'characteristics_raw', 'source_name'])

In [6]:
len(norms)

376742

In [10]:
norms_df = pd.DataFrame(dict(accession=norms))
norms_df.head()

,accession
0,GSM1422840
1,GSM1422847
2,GSM1422852
3,GSM1422846
4,GSM1422851


In [11]:
norms_df.to_pickle('../data/geo-annotation/norms.v1.pickle')

In [7]:
norms_df = pd.read_pickle('../data/geo-annotation/norms.v1.pickle')

In [13]:
samples_series = (
    pd.read_pickle('../data/geo-annotation/samples_series.v1.pickle')
    .rename(columns={0: 'accession', 1: 'series'})
)

In [14]:
samples_series.head()

,accession,series
0,GSM1,GSE506
1,GSM2,GSE506
2,GSM3,GSE462
3,GSM4,GSE462
4,GSM5,GSE462


In [ ]:
s

In [11]:
norms_df['norm'] = 1
norms_df.head()

,accession,norm
0,GSM1422840,1
1,GSM1422847,1
2,GSM1422852,1
3,GSM1422846,1
4,GSM1422851,1


In [15]:
samples_with_norms = (
    pd.merge(norms_df, samples_series, how='right')
    .drop_duplicates(subset=['accession', 'series'])
    .fillna(0)
)
print(samples_with_norms.shape)
samples_with_norms.head()

(2425973, 3)


,accession,norm,series
0,GSM1422840,1,GSE58949
1,GSM1422847,1,GSE58949
2,GSM1422852,1,GSE58949
3,GSM1422846,1,GSE58949
4,GSM1422851,1,GSE58949


## Norms count in dataset analysis

In [17]:
samples_with_norms.series.unique().shape

(66842,)

In [16]:
(
    samples_with_norms
    .groupby('series')
    .agg(dict(norm=lambda norms: sum(norms)/len(norms), accession=len))
    .query('norm > 0')
#     .accession
#     .sum()
    .shape
)

(21017, 2)

In [20]:
(
    samples_with_norms
    .groupby('series')
    .agg(dict(norm=lambda norms: sum(norms)/len(norms), accession=len))
    .query('0 < norm < 0.8')
    .shape
)

(15191, 2)

In [29]:
shapes = [(
    samples_with_norms
    .groupby('series')
    .agg(dict(norm=lambda norms: sum(norms)/len(norms), accession=len))
    .query('0 < norm < {}'.format(float(i/10)))
    .shape[0]
) for i in range(0, 11, 1)]

In [30]:
shapes

[(0, 2),
 (797, 2),
 (2267, 2),
 (4590, 2),
 (7033, 2),
 (8191, 2),
 (14285, 2),
 (14878, 2),
 (15191, 2),
 (15362, 2),
 (15448, 2)]

In [34]:
good_series = (
    samples_with_norms
    .groupby('series')
    .agg(dict(norm=lambda norms: sum(norms)/len(norms), accession=len))
    .query('0 < norm < 0.7')
    .index
)

In [39]:
(
    samples_with_norms
    [samples_with_norms.series.isin(good_series)]
    .drop('series', axis=1)
    .drop_duplicates()
#     .shape
    .to_pickle('../data/geo-annotation/samples.norms.07.res4.pickle')
)

In [40]:
# samples_with_norms = pd.read_pickle('../data/geo-annotation/samples.norms.res3.pickle')